In [1]:
import json

from pathlib import Path

from pydantic import parse_obj_as

from kptncook.config import settings
from kptncook.models import Recipe

from kptncook.mealie import MealieApiClient, Unit, kptncook_to_mealie

# Use Example Recipe

In [2]:
example_path = Path.cwd().parent / "tests" / "fixtures" / "kptncook_example.json"
with example_path.open("r") as f:
    example = json.load(f)
recipe = Recipe.parse_obj(example)

In [3]:
measures = set(filter(None, [i.measure for i in recipe.ingredients]))

In [4]:
measures

{'EL', 'Zehe(n)', 'g'}

In [10]:
class NewClient(MealieApiClient):
    def _get_page(self, endpoint_name, page_num, per_page=50):
        r = self.get(f"/{endpoint_name}?page={page_num}&perPage={per_page}")
        r.raise_for_status()
        return r.json()
    
    def _get_all_items(self, endpoint_name):
        all_items = []        
        response_data = self._get_page(endpoint_name, 1)
        all_items.extend(response_data["items"])
        
        # 1 was already fetched, start page_num at 2 and add 1 to the
        # number of total pages, because we start counting at 1 instead of 0
        for page_num in range(2, response_data["total_pages"] + 1):
            response_data = self._get_page(endpoint_name, page_num)
            all_items.extend(response_data["items"])
        
        return all_items

    def _create_item(self, endpoint_name, item):
        r = self.post(f"/{endpoint_name}", data=item.json())
        r.raise_for_status()
        return r.json()
    
    def _create_unit_name_to_unit_lookup(self, recipe_units):
        existing_units = parse_obj_as(set[Unit], self._get_all_items("units"))
        units_to_create = recipe_units - existing_units
        for unit in units_to_create:
            existing_units.add(Unit(**client._create_item("units", unit)))
        return {u.name: u for u in existing_units}
    
    def _update_unit_ids(self, recipe):
        recipe_units = {ig.unit for ig in mealie_recipe.recipe_ingredient if ig.unit is not None}
        if len(recipe_units) == 0:
            # return early if there's nothing to do
            return recipe
        
        name_to_unit_with_id = self._create_unit_name_to_unit_lookup(recipe_units)
        for ingredient in recipe.recipe_ingredient:
            if ingredient.unit is not None:
                ingredient.unit = name_to_unit_with_id[ingredient.unit.name]
        return recipe

In [11]:
client = NewClient("http://localhost:9000/api")
client.login(username="jochen")

 ········


In [12]:
mealie_recipe = kptncook_to_mealie(recipe)

In [13]:
mealie_recipe = client._update_unit_ids(mealie_recipe)

In [16]:
foo = [ig.unit for ig in mealie_recipe.recipe_ingredient if ig.unit is not None][0]

In [17]:
hash(foo)

-5452980252907157313

In [19]:
hash(foo.name)

-5452980252907157313